# インポート

In [11]:
from pathlib import Path
import json

## issue_numberを取得

In [20]:
from pathlib import Path
import json

def read_issue_number():
    """
    issue_numberをjsonで取得する関数
    """
    # このファイル自身の場所（scripts/）
    base_dir = Path(__file__).resolve().parent
    # /boatrace/db/scrape_race_schedule_dict.json
    json_path = base_dir.parent / "db" / "issue_number.json"
    with open(json_path, "r", encoding="utf-8") as f:
        issue_number_list = json.load(f)
    return issue_number_list

## 

In [13]:
import urllib.request as req

url = "https://takarakuji.rakuten.co.jp/backnumber/loto6/202601/"
html = req.urlopen(url).read()

In [15]:
import pandas as pd

pd.read_html(html, encoding="utf-8")

[         回号        第2065回      第2065回.1      第2065回.2      第2065回.3  \
 0      抽せん日    2026/01/05    2026/01/05    2026/01/05    2026/01/05   
 1       本数字             6            18            21            31   
 2    ボーナス数字           (3)           NaN           NaN           NaN   
 3        1等          該当なし          該当なし          該当なし          該当なし   
 4        2等            5口            5口            5口   15,198,300円   
 5        3等          215口          215口          215口      381,700円   
 6        4等       10,741口       10,741口       10,741口        8,000円   
 7        5等      178,338口      178,338口      178,338口        1,000円   
 8  キャリーオーバー  259,218,384円  259,218,384円  259,218,384円  259,218,384円   
 
        第2065回.4      第2065回.5  
 0    2026/01/05    2026/01/05  
 1            37            40  
 2           NaN           NaN  
 3          該当なし          該当なし  
 4   15,198,300円   15,198,300円  
 5      381,700円      381,700円  
 6        8,000円        8,000円  
 7        1,00

In [26]:
import json

with open("../db/issue_number.json", "rb") as f:
    issue_number_list = json.load(f)

In [27]:
issue_number_list

['202601',
 '202512',
 '202511',
 '202510',
 '202509',
 '202508',
 '202507',
 '202506',
 '202505',
 '202504',
 '202503',
 '202502',
 '1961-1969',
 '1941-1960',
 '1921-1940',
 '1901-1920']

In [29]:
from pathlib import Path

HTML_DIR = Path("..", "data", "html")
HTML_DIR

PosixPath('../data/html')

In [35]:
import time

for issue_number in issue_number_list:
    if "-" not in issue_number:
        url = f"https://takarakuji.rakuten.co.jp/backnumber/loto6/{issue_number}/"
    else:
        url = f"https://takarakuji.rakuten.co.jp/backnumber/loto6_detail/{issue_number}/"
    html = req.urlopen(url).read()
    time.sleep(1)
    with open(HTML_DIR / f"{issue_number}.bin", "wb") as f:
        f.write(html)

In [48]:
from tqdm.notebook import tqdm

def write_html(issue_number_list, save_dir: Path):
    html_path_list = []
    for issue_number in tqdm(issue_number_list):
        filepath = save_dir / f"{issue_number}.bin"
        # binファイルがすでに存在する場合はスキップする
        if filepath.is_file():
            print(f"skipped: {issue_number}")
        else:
            if "-" not in issue_number:
                url = f"https://takarakuji.rakuten.co.jp/backnumber/loto6/{issue_number}/"
            else:
                url = f"https://takarakuji.rakuten.co.jp/backnumber/loto6_detail/{issue_number}/"
            html = req.urlopen(url).read()
            time.sleep(1)
            with open(filepath, "wb") as f:
                f.write(html)
            html_path_list.append(filepath)
    return html_path_list

In [49]:
html_path_list = write_html(issue_number_list, save_dir=HTML_DIR)

  0%|          | 0/16 [00:00<?, ?it/s]

skipped: 202601
skipped: 202512
skipped: 202511
skipped: 202510
skipped: 202509
skipped: 202508
skipped: 202507
skipped: 202506
skipped: 202505
skipped: 202504
skipped: 202503
skipped: 202502
skipped: 1961-1969
skipped: 1941-1960
skipped: 1921-1940
skipped: 1901-1920


In [51]:
html_path_list

[]

In [56]:
html_path_list = list(HTML_DIR.glob("*"))
html_path_list

[PosixPath('../data/html/202601.bin'),
 PosixPath('../data/html/1921-1940.bin'),
 PosixPath('../data/html/202502.bin'),
 PosixPath('../data/html/202503.bin'),
 PosixPath('../data/html/202507.bin'),
 PosixPath('../data/html/202506.bin'),
 PosixPath('../data/html/202512.bin'),
 PosixPath('../data/html/202504.bin'),
 PosixPath('../data/html/202510.bin'),
 PosixPath('../data/html/202511.bin'),
 PosixPath('../data/html/202505.bin'),
 PosixPath('../data/html/202508.bin'),
 PosixPath('../data/html/1961-1969.bin'),
 PosixPath('../data/html/202509.bin'),
 PosixPath('../data/html/1941-1960.bin'),
 PosixPath('../data/html/1901-1920.bin')]

In [178]:
processed_dfs = {}
for html_path in html_path_list[:2]:
    with open(html_path, "rb") as f:
        html = f.read()
        dfs = pd.read_html(html)
        count = 0
        indexes = []

        for i, df in enumerate(dfs):
            if df.astype(str).apply(lambda x: x.str.contains("抽せん日")).any().any():
                count += 1
                indexes.append(i)
        target_dfs = [dfs[i] for i in indexes]


        for df in target_dfs:
            df2 = df.drop(index=range(3, 9), errors="ignore")
            number_id = df2.columns[1]
            df2.index = [number_id] * len(df2)
            df2.columns = [0, 1, 2, 3, 4, 5, 6]
            processed_dfs[number_id] = df2
concat_df = pd.concat(processed_dfs.values())
concat_df.index.name = "回号"

In [ ]:
concat_df

In [101]:
import pandas as pd
import urllib.request as req
from bs4 import BeautifulSoup

url = "https://takarakuji.rakuten.co.jp/backnumber/loto6/202601/"
html = req.urlopen(url).read()
dfs = pd.read_html(html, encoding="utf-8")

count = 0
indexes = []

for i, df in enumerate(dfs):
    if df.astype(str).apply(lambda x: x.str.contains("抽せん日")).any().any():
        count += 1
        indexes.append(i)
target_dfs = [dfs[i] for i in indexes]
target_dfs

[         回号        第2065回      第2065回.1      第2065回.2      第2065回.3  \
 0      抽せん日    2026/01/05    2026/01/05    2026/01/05    2026/01/05   
 1       本数字             6            18            21            31   
 2    ボーナス数字           (3)           NaN           NaN           NaN   
 3        1等          該当なし          該当なし          該当なし          該当なし   
 4        2等            5口            5口            5口   15,198,300円   
 5        3等          215口          215口          215口      381,700円   
 6        4等       10,741口       10,741口       10,741口        8,000円   
 7        5等      178,338口      178,338口      178,338口        1,000円   
 8  キャリーオーバー  259,218,384円  259,218,384円  259,218,384円  259,218,384円   
 
        第2065回.4      第2065回.5  
 0    2026/01/05    2026/01/05  
 1            37            40  
 2           NaN           NaN  
 3          該当なし          該当なし  
 4   15,198,300円   15,198,300円  
 5      381,700円      381,700円  
 6        8,000円        8,000円  
 7        1,00

In [102]:
processed_dfs = []

for df in target_dfs:
    df2 = df.drop(index=range(3, 9), errors="ignore")
    processed_dfs.append(df2)
processed_dfs[0]

,回号,第2065回,第2065回.1,第2065回.2,第2065回.3,第2065回.4,第2065回.5
0,抽せん日,2026/01/05,2026/01/05,2026/01/05,2026/01/05,2026/01/05,2026/01/05
1,本数字,6,18,21,31,37,40
2,ボーナス数字,(3),NaN,NaN,NaN,NaN,NaN


In [95]:
race_id = processed_dfs[0].columns[1]   # 第2065回

df2 = processed_dfs[0].copy()
df2.columns = df2.iloc[0]
df2 = df2.iloc[1:].reset_index(drop=True)
df2.index = [race_id] * len(df2)
df2


,抽せん日,2026/01/05,2026/01/05,2026/01/05,2026/01/05,2026/01/05,2026/01/05
第2065回,本数字,6,18,21,31,37,40
第2065回,ボーナス数字,(3),NaN,NaN,NaN,NaN,NaN


In [67]:
url = "https://takarakuji.rakuten.co.jp/backnumber/loto6/202601/"
html = req.urlopen(url).read()
dfs = pd.read_html(html, encoding="utf-8")
dfs[0]

,回号,第2065回,第2065回.1,第2065回.2,第2065回.3,第2065回.4,第2065回.5
0,抽せん日,2026/01/05,2026/01/05,2026/01/05,2026/01/05,2026/01/05,2026/01/05
1,本数字,6,18,21,31,37,40
2,ボーナス数字,(3),NaN,NaN,NaN,NaN,NaN
3,1等,該当なし,該当なし,該当なし,該当なし,該当なし,該当なし
4,2等,5口,5口,5口,"15,198,300円","15,198,300円","15,198,300円"
5,3等,215口,215口,215口,"381,700円","381,700円","381,700円"
6,4等,"10,741口","10,741口","10,741口","8,000円","8,000円","8,000円"
7,5等,"178,338口","178,338口","178,338口","1,000円","1,000円","1,000円"
8,キャリーオーバー,"259,218,384円","259,218,384円","259,218,384円","259,218,384円","259,218,384円","259,218,384円"
